In [23]:
import tensorflow as tf
import numpy as np

In [24]:
# [털, 날개]
x_data = np.array([[0,0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])
y_data = np.array([
    [1, 0, 0], # 기타
    [0, 1, 0], # 포유류
    [0, 0, 1], # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

In [25]:
# 학습에 직접 사요하지 않고 학습 횟수를 카운트하기 위해 사용함
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2, 10], -1.0, 1.0))
b1 = tf.Variable(tf.zeros([10]))
L1 = tf.add(tf.matmul(X, W1), b1)

W2 = tf.Variable(tf.random_uniform([10, 20], -1.0, 1.0))
b2 = tf.Variable(tf.zeros([20]))
L2 = tf.add(tf.matmul(L1, W2), b2)

W3 = tf.Variable(tf.random_uniform([20, 3], -1.0, 1.0))
b3 = tf.Variable(tf.zeros([3]))

model = tf.add(tf.matmul(L2, W3), b3)

In [26]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost, global_step=global_step)

# 모델 학습

In [38]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model/5')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

INFO:tensorflow:Restoring parameters from ./model/5/ckpt-10


In [39]:
for step in range(2):
    sess.run(train_op, feed_dict={X:x_data, Y:y_data})
    
    print('Step : %d ' % sess.run(global_step))
    print('Cost : %.3f ' % sess.run(cost, feed_dict={X: x_data, Y:y_data}))

saver.save(sess, './model/5/ckpt', global_step=global_step)

Step : 11 
Cost : 0.082 
Step : 12 
Cost : 0.065 


'./model/5/ckpt-12'

# 결과 확인

In [40]:
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도 : %.2f' % sess.run(accuracy*100, feed_dict = {X: x_data, Y: y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도 : 100.00
